In [5]:
import pickle
import pandas as pd
import os

In [2]:
#import haystack and FARM utils
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

#initialize elasticsearch docker image
! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.6.2

30e249ac9ff112aba5592be72ea3ca19f9221965eb424555810e762c1cd6bcca


In [3]:
#initialize document storage
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

10/03/2020 09:55:54 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.321s]
10/03/2020 09:55:54 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.241s]


In [6]:
#import data
'''processed log-frames'''
with open(os.path.abspath(os.path.join('..', 'data'))+'/logframes_clean.pkl', 'rb') as handle:
    data = pickle.load(handle)

In [7]:
splitted = pd.concat([pd.Series(row['PIMS_ID'], row['full_obj_or_outcome'].split("',"), )              
                    for _, row in data.iterrows()]).reset_index()

splitted = splitted.rename(columns={"index": "text", 0: "PIMS_ID"})

In [43]:
dict_list = []
for i, row in splitted.iterrows():
    write_dicts = {'text': row.text, 'PIMS_ID': row.PIMS_ID}
    dict_list.append(write_dicts)
dicts = tuple(dict_list)

In [46]:
document_store.write_documents(dicts)

10/03/2020 10:04:52 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.585s]
10/03/2020 10:04:53 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.043s]
10/03/2020 10:04:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.035s]
10/03/2020 10:04:55 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.027s]
10/03/2020 10:04:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.098s]
10/03/2020 10:04:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.981s]
10/03/2020 10:04:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.017s]
10/03/2020 10:04:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.076s]


10/03/2020 10:06:01 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.001s]
10/03/2020 10:06:02 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.002s]
10/03/2020 10:06:03 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.755s]
10/03/2020 10:06:04 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.014s]
10/03/2020 10:06:05 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.998s]
10/03/2020 10:06:06 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.002s]
10/03/2020 10:06:07 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.003s]
10/03/2020 10:06:08 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.981s]


In [50]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [51]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

10/03/2020 10:07:54 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
10/03/2020 10:07:54 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
10/03/2020 10:07:55 - INFO - filelock -   Lock 5161073776 acquired on /Users/jonas/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock


10/03/2020 10:07:56 - INFO - filelock -   Lock 5161073776 released on /Users/jonas/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock


10/03/2020 10:07:56 - INFO - filelock -   Lock 5296399120 acquired on /Users/jonas/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


10/03/2020 10:11:01 - INFO - filelock -   Lock 5296399120 released on /Users/jonas/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


10/03/2020 10:11:07 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
10/03/2020 10:11:14 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
10/03/2020 10:11:16 - INFO - filelock -   Lock 5297618032 acquired on /Users/jonas/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


...

10/03/2020 10:11:22 - INFO - filelock -   Lock 5297713792 acquired on /Users/jonas/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


10/03/2020 10:11:23 - INFO - filelock -   Lock 5297713792 released on /Users/jonas/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


10/03/2020 10:11:24 - INFO - filelock -   Lock 5297703904 acquired on /Users/jonas/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


10/03/2020 10:11:25 - INFO - filelock -   Lock 5297703904 released on /Users/jonas/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


10/03/2020 10:11:26 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
10/03/2020 10:11:26 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
10/03/2020 10:11:26 - INFO - farm.infer -    0    0    0 
10/03/2020 10:11:26 - INFO - farm.infer -   /w\  /w\  /w\
10/03/2020 10:11:26 - INFO - farm.infer -   /'\  / \  /'\
10/03/2020 10:11:26 - INFO - farm.infer -       


In [52]:
finder = Finder(reader, retriever)

In [62]:
prediction = finder.get_answers(question="what does mining do to the environment", top_k_retriever=10, top_k_reader=5)

10/03/2020 10:16:12 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.065s]
10/03/2020 10:16:12 - INFO - haystack.retriever.sparse -   Got 10 candidates from retriever
10/03/2020 10:16:12 - INFO - haystack.finder -   Reader is looking for detailed answer in 3746 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.66 Batches/s]


In [63]:
print_answers(prediction)

{   'answers': [   {   'answer': 'negative impacts on grasslands biodiversity',
                       'context': ' has seen an increasing expansion of coal '
                                  'mining with negative impacts on grasslands '
                                  'biodiversity. Unfortunately DME does not '
                                  'use the MBCP to guide th',
                       'document_id': '70dd8aa2-f187-40db-ba50-4e0a1b4e60ae',
                       'meta': {'PIMS_ID': '2929'},
                       'offset_end': 97,
                       'offset_end_in_doc': 486,
                       'offset_start': 54,
                       'offset_start_in_doc': 443,
                       'probability': 0.7538280818426594,
                       'score': 8.953073501586914},
                   {   'answer': 'Number of agro-pastoralists having applied '
                                 'sustainable intensification techniques',
                       'context': 'soil\'s 

In [59]:
data.loc[data['PIMS_ID'] == "5874"]

,PIMS_ID,lead_country,description,log_frame,technical_team,focal_area,signature_programmes,full_title,keywords,region,...,sourcesOfFunds,participatingCountries,projectSectors,resultAreas,jointAgencies,objectives,outcomes,full_obj_or_outcome,language,text
24,5874,Peru,,[{'indicator': [{'end_of_project_target_level': '4 new partnership mechanism...,Chemicals,Chemicals and Waste,['SP 1. Reducing POPs and mercury releases through the sound management of c...,GEF GOLD Peru - Integrated Sound Management of Mercury in Peru’s Artisanal a...,[],Latin America & the Caribbean,...,['GEF Trust Fund'],['Peru'],[],[],['UN Environment'],['4 new partnership mechanisms with funding for gender friendly and sustaina...,['Capacity of 4 government entities increased to improve their capacity to a...,['4 new partnership mechanisms with funding for gender friendly and sustaina...,en,new partnership mechanism funding gender friendly sustainable management sol...


In [ ]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  use_gpu=True,
                                  embed_title=True,
                                  max_seq_len=512,
                                  batch_size=16,
                                  remove_sep_tok_from_untitled_passages=True)

10/03/2020 10:28:53 - INFO - filelock -   Lock 6314954224 acquired on /Users/jonas/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock
